In [273]:
#Heap stuff 
type Node
    # a node data structure. It contains two arguments: a key and a value. "key" 
    # is the used to rank the node, possibly by a heap. "val" is the value of the 
    # node from the perspective of its graph.
    key::Real
    val::Real
end

type Heap
    # a heap data structure. It contains a list of node objects. Note that elements 
    # are never deleted from the node list; rather, a pointer called "last" is used 
    # to indicate that last good value in the arrays. The heaps can be either min or 
    # max heaps, as determined by the keyword argument "kind"
    list::Array{Node,1}
    last::Int
    kind::ASCIIString
end
Heap(kind) = Heap([], 0, kind)
Heap(node, kind) = Heap([node], 1, kind)
 
function sink(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) of the node to be sunk can be found 
    # what it does:
    #     after the base node is popped from the heap, the node in the last 
    #     position is brough into the base node, at least temporarily. The proper 
    #     position for this node can be found by "sinking" it down through the heap.
    # returns:
    #    return statements are for facilitation of recursion only
    if heap.last != 0
        if heap.kind == "min"
            ∘ = <=
        elseif heap.kind == "max"
            ∘ = >=
        end
        l = i<<1
        r = i<<1+1
        if (r <= heap.last && 
                heap.list[r].key ∘ heap.list[l].key && 
                heap.list[r].key ∘ heap.list[i].key)
            heap.list[i], heap.list[r] = heap.list[r], heap.list[i]
            return sink(heap, r)
        elseif l <= heap.last && heap.list[l].key ∘ heap.list[i].key
            heap.list[i], heap.list[l] = heap.list[l], heap.list[i]
            return sink(heap, l)
        else
            return
        end
    end
end

function float(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) of the node to be floated can be found 
    # what it does:
    #     a new node is pushed onto the heap in the last position. This may not be 
    #     its rightful location in the heap. The proper position for this node can 
    #     be found by "floating" it up through the heap.
    # returns:
    #    return statements are for facilitation of recursion only
    if i == 1
        return
    else
        if heap.kind == "min"
            ∘ = <
        elseif heap.kind == "max"
            ∘ = >
        end
        if heap.list[i].key ∘ heap.list[i>>1].key
            heap.list[i], heap.list[i>>1] = heap.list[i>>1], heap.list[i]
            return float(heap, i>>1)
        end
    end
end
        
function pop(heap)
    # arguments:
    #     heap: a heap data structure
    # what it does:
    #     removes the base node from the heap. Note that elements are never 
    #     deleted from the heap's lists, rather the last pointer is decremented
    # returns:
    #    the contents of the base node: its key and value. If the heap is empty 
    #    it returns an infinity-stuffed node
    if heap.last > 0
        returnable = heap.list[1]
        heap.list[1] = heap.list[heap.last]
        heap.last -= 1
        sink(heap, 1)
        return returnable
    else
        return Node(Inf,Inf)
    end
end

function push(heap, nuno)
    # arguments:
    #     heap: a heap data structure
    #     key: the quantity of the new node around which the heap is organized
    #     value: other quantity associated with the node but unimportant to the heap
    # what it does:
    #     adds a new node to the heap. Note that if the heap's lists are too short to 
    #     contain the new nodes children, dummy elements are added to the lists. The 
    #     keys of these dummies are the maximum allowed by the data type if a min heap, 
    #     zero if a max heap. This is done to facilitate sinking after pop.
    # returns:
    #    no returns
    heap.last += 1
    if size(heap.list)[1] < heap.last
        push!(heap.list, nuno)
    else
        heap.list[heap.last] = nuno
    end
    float(heap, heap.last)
end


push (generic function with 1 method)

In [274]:
function BFS(graph, heap, shortestpaths, explored)
    # arguments:
    #     graph: an adjacency matrix of an undirected graph of the form 
    #     Dict{Any,Dict{Any,Number}}.
    #     heap: a heap data structure with graph nodes as elements
    #     shortestpaths: a (partial) list of nodes in increasomg order of 
    #     shortest path
    #     explored: a list of booleans corresponding to the visitation status of 
    #     the node corresponding to the index of the array
    # what it does:
    #     Pops the top off the heap and adds a new nodes to the frontier--all 
    #     nodes which are adjacent to the node at the top of the heap and that 
    #     have not been previously explored. Nodes are not considered explored until 
    #     they have been popped off the heap. 
    # returns:
    #    the argument heap with new nodes to be explored and the base node popped off
    #    an argument shortestpaths list updated with the shortest remaining path 
    #    from the heap
    #    the argument explored list with the most recently explored node included
    current = pop(heap)
    if current.val != Inf
        if !explored[current.val]
            push!(shortestpaths, current)
            pathlength = current.key
            explored[current.val] = true
            for k in keys(graph[current.val])
                if !explored[k]
                    push(heap, Node(pathlength + graph[current.val][k], k))
                end
            end
        end
    else
        explored = ones(Bool, size(explored)[1])
    end
    return heap, shortestpaths, explored
end

function Dijkstra(graph, startnode)
    # arguments:
    #     graph: an undirected adjacency list of the form Dict{Any,Dict{Any,Number}}.
    #     startnode: the starting point from which all shortest paths will be measured
    # what it does:
    #     finds the shortest path to all points from the startnode
    # returns:
    #    a complete list of nodes sorted in order of node number. Each node ion the 
    #    list contains the shortest path as its "key", and the node number as its "val"
    shortestpaths = []
    explored = [false for i = 1:maximum(keys(graph))]
    heap = Heap(Node(0, startnode), "min")
    while !all(explored)
        heap, shortestpaths, explored = BFS(graph, heap, shortestpaths, explored)
    end
    return shortestpaths
end

Dijkstra (generic function with 1 method)

# An Example


This data which is loaded in the cell below is the proper form of the graph to feed to the Dijkstra algorithm. The structure must be of the form Dict{Int, Dict{Int, Real}}. Interpret its meaning through the following example, that of node 1. There are a number of undirected edges leading out of it, including edges to node 169 with edge length 2001 and node 11 with edge length 70.

In [2]:
using JLD
adj = load("dijkstra.jld", "al") 

adj[1]

Dict{Int64,Real} with 15 entries:
  169 => 2001
  11  => 70
  91  => 2446
  26  => 2838
  10  => 1918
  138 => 7783
  156 => 5786
  177 => 3779
  61  => 8958
  78  => 6878
  154 => 2697
  67  => 11
  36  => 8196
  164 => 7749
  33  => 5564

Here are the results of the test run. The nodes are printed in order of shortest path length fron Node 1.

In [277]:
@time Dijkstra(adj, 1)

  0.006739 seconds (7.39 k allocations: 191.188 KB)


200-element Array{Any,1}:
 Node(0,1)     
 Node(11,67)   
 Node(70,11)   
 Node(72,72)   
 Node(168,94)  
 Node(450,73)  
 Node(594,196) 
 Node(724,130) 
 Node(770,133) 
 Node(778,136) 
 Node(815,3)   
 Node(864,74)  
 Node(897,160) 
 ⋮             
 Node(3807,76) 
 Node(3860,40) 
 Node(3861,90) 
 Node(3997,184)
 Node(4102,31) 
 Node(4127,45) 
 Node(4226,173)
 Node(4348,141)
 Node(4357,95) 
 Node(4629,168)
 Node(4950,70) 
 Node(5403,163)